# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [92]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [32]:
df = pd.read_csv('data/Admission_Predict.csv')
df = pd.DataFrame(df)
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [33]:
df.columns = df.columns.str.rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [34]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [35]:
df.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [36]:
df.set_index('Serial No.', inplace=True)

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [37]:
df[['GRE Score', 'CGPA']]

,GRE Score,CGPA
Serial No.,,
1,337,9.65
2,316,8.00
3,322,8.67
4,314,8.21
5,330,9.34
...,...,...
381,324,9.04
382,325,9.11
383,330,9.45


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [38]:
#df = df.set_index(['GRE Score', 'CGPA'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,316,104,3,3.0,3.5,8.00,1,0.72
2,322,110,3,3.5,2.5,8.67,1,0.80
3,314,103,2,2.0,3.0,8.21,0,0.65
4,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
380,324,110,3,3.5,3.5,9.04,1,0.82
381,325,107,3,3.0,3.5,9.11,1,0.84
382,330,116,4,5.0,4.5,9.45,1,0.91
383,312,103,3,3.5,4.0,8.78,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [47]:
df.loc[(df["CGPA"] > 9) & (df["Research"] >= 1)].shape

#101 rows

(101, 8)

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [65]:
df.loc[(df["CGPA"] > 9) & (df["SOP"] < 3.5)].describe().head(2)

#0.802

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,5.0,5.0,5.0,5.0,5.0,5.00,5.0,5.000
mean,328.0,112.8,3.4,3.0,3.4,9.13,0.8,0.802


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [67]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    return (col - np.mean(col)) / np.std(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [83]:
df['CGPA_std'] = standardize(df['CGPA'])
df['GRE_std'] = standardize(df['GRE Score'])
df['LOR_std'] = standardize(df['LOR'])

We will generate the decision choice at random using the code below. Please run the cell.

In [84]:
# Libraries
from random import choices

In [85]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])
decision_choice

['GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [86]:
df['decision_choice'] = df.lookup(df.index, decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [88]:
df['decision'] = df['decision_choice'].apply(lambda x: 1 if x >= 0.8 else 0)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [91]:
df['decision'].value_counts()

#85 will be accepted

0    300
1     85
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [99]:
df.columns = [re.sub('[\W]', '', re.sub(' ', '_', char)).lower() for char in df.columns]
df

,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,decision_choice,decision
0,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0
2,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.076840,0
381,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.076840,0
382,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197,1
383,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [118]:
df['adjusted_gre'] = pd.cut(df[df['university_rating'] >= 4].apply(lambda x: x['gre_score'] + 10, axis=1), bins = 4)

In [119]:
df

,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,decision_choice,decision,adjusted_gre
0,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663,1,"(337.5, 350.0]"
1,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0,NaN
2,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0,NaN
3,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0,NaN
4,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1,"(337.5, 350.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.076840,0,NaN
381,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.076840,0,NaN
382,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197,1,"(337.5, 350.0]"
383,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,0,NaN
